In [ ]:
import torch
import torchvision
import cv2
import numpy as np
from sort.sort import *
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
obt = DeepSort(max_iou_distance=1, max_age=200,nms_max_overlap = 5)
pic = cv2.imread('cars.jpg')
predictions = model(pic)
car_predictions = predictions.pred[0][predictions.pred[0][:, 5] == 2]
b = car_predictions[:, :4].cpu().numpy()
c = car_predictions[:, 4].cpu().numpy()
lbl = car_predictions[:, 5].cpu().numpy()
for box, confidence in zip(b, c):
    if confidence > 0.5:
        x1, y1, x2, y2 = box.astype(int)
        cv2.rectangle(pic, (x1, y1), (x2, y2), (0, 255, 0), 2)
cv2.imshow('image', pic)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def detect_cars(pic, model, threshold = 0.5):
    dts = []
    pred = model(pic)
    car_pred = pred.pred[0][pred.pred[0][:, 5] == 2]
    b = car_pred[:, :4].cpu().numpy()
    c = car_pred[:, 4].cpu().numpy()
    lbl = car_pred[:, 5].cpu().numpy()
    for i in range(len(lbl)):
        if(c[i] > threshold):
            cv2.rectangle(pic, (int(b[i][0]), int(b[i][1])), (int(b[i][2]), int(b[i][3])), (0, 255, 0), 2)
            cv2.putText(pic, "Car", (int(b[i][0])-10, int(b[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            dts.append((b[i], c[i]))    
    return dts

In [ ]:
def update_track(tracks, count, IDs):
    for t in tracks:
        if not t.is_confirmed():
            continue
        trck = t.track_id
        if(trck not in IDs):
            count += 1
            IDs.append(trck)        
        ltrb = t.to_ltrb()
        bbox = ltrb
    return IDs, count

In [ ]:
cap = cv2.VideoCapture('./mp4/Test.mp4')

IDs = []
count = 0 

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    dts = detect_cars(frame, model, 0.5)
    tracks = obt.update_tracks(dts, frame=frame)
    IDs, count = update_track(tracks, count, IDs)
    cv2.putText(frame, f'Count: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()